# The Compression Abaqus Model Test

## Import modules

Usually, when we use Python script to build the model, we need to import the following modules:

In [1]:
from abaqus import *
from abaqusConstants import *
from caeModules import *
from driverUtils import *

Module `abaqus` contains two most important variable `mdb` and `session` which control the model. Module `abaqusConstants` contains constant strings used in modelling.

Module `driverUtils` contains an important function `executeOnCaeStartup`, this function will be executed each time we open the Abaqus, so we need to call this function in our Python script.

In [2]:
executeOnCaeStartup()

## The model object

The `mdb` object is the high-level Abaqus model database. A model database stores models and analysis controls.  A model database stores models and analysis controls. The most important object is the `model` object.

In [3]:
model = mdb.models['Model-1']

## Create parts

First we need to create a sketch that will be used to create the part, we need to use `ConstrainedSketch()` to create a sketch:

In [4]:
sketch = model.ConstrainedSketch(name='sketch', sheetSize=1.0)
sketch.rectangle((0, 0), (1, 1))

In this code, we draw a sketch with a square. Now we can create a part using this sketch:

In [5]:
part = model.Part(name='part', dimensionality=THREE_D, type=DEFORMABLE_BODY)
part.BaseSolidExtrude(sketch=sketch, depth=1)

The first line creates a 3D and deformable part. Then we use the BaseSolidExtrude() method to create a part using the sketch.

## Create some sets for boundary conditions and loads

Unlike building a model in Abaqus/CAE, we can just click the nodes/faces to create sets, when we use a Python script to build the model, we can use coordinates to find nodes/faces we need.

We can use `Set()` and `Surface()` to create sets and surfaces:

In [6]:
part.Set(name='set-all', cells=part.cells.findAt(coordinates=((0.5, 0.5, 0.5), )))
part.Set(name='set-bottom', faces=part.faces.findAt(coordinates=((0.5, 0.5, 0.0), )))
part.Set(name='set-top', faces=part.faces.findAt(coordinates=((0.5, 0.5, 1.0), )))
part.Surface(name='surface-top', side1Faces=part.faces.findAt(coordinates=((0.5, 0.5, 1.0), )))

## Merge parts to assembly

We can use Instance() to create instances：

In [7]:
model.rootAssembly.DatumCsysByDefault(CARTESIAN)
model.rootAssembly.Instance(name='instance', part=part, dependent=ON)

## Create materials and sections, and assign materials to sections

First we create a Material object using `Material()`:

In [8]:
material = model.Material(name='material')

Then we assign some properties to the Material object, i.e., `Elastic()` and `Density()`:

In [9]:
material.Elastic(table=((1000, 0.2), ))
material.Density(table=((2500, ), ))

Then we create a `HomogeneousSolidSection()` and assign the material to the section (`SectionAssignment()`):

In [10]:
model.HomogeneousSolidSection(name='section', material='material', thickness=None)
part.SectionAssignment(region=part.sets['set-all'], sectionName='section')

## Create steps

It is easy to create a `StaticStep()`:

In [11]:
step = model.StaticStep(name='Step-1', previous='Initial', description='',
                        timePeriod=1.0, timeIncrementationMethod=AUTOMATIC,
                        maxNumInc=100, initialInc=0.01, minInc=0.001, maxInc=0.1)

## Specify output requests

We can use the `FieldOutputRequest()` and `HistoryOutputRequest()` to specify field output and history output information.

In [12]:
field = model.FieldOutputRequest('F-Output-1', createStepName='Step-1',
                                 variables=('S', 'E', 'U'))

## Create boundary conditions

We can use `DisplacementBC()` to create a displacement boundary condition:

In [13]:
bottom_instance = model.rootAssembly.instances['instance'].sets['set-bottom']
bc = model.DisplacementBC(name='BC-1', createStepName='Initial',
                          region=bottom_instance, u3=SET)

It should be noted that region of the boundary condition should be a region of the instances instead of parts, since sets created in parts are copied to the instance, we can use the sets in the parts that we defined before.

## Create loads

We can use `Pressure()` ro create a pressure:

In [14]:
top_instance = model.rootAssembly.instances['instance'].surfaces['surface-top']
pressure = model.Pressure('pressure', createStepName='Step-1', region=top_instance,
                          magnitude=100)

## Mesh

To mesh the model, we have to set the `ElemType`, which is defined in the mesh module, so we need to import mesh module:

In [15]:
import mesh

elem1 = mesh.ElemType(elemCode=C3D8R)
elem2 = mesh.ElemType(elemCode=C3D6)
elem3 = mesh.ElemType(elemCode=C3D4)
part.setElementType(regions=(part.cells, ), elemTypes=(elem1, elem2, elem3))
part.seedPart(size=0.1)
part.generateMesh()

## Create jobs

We can use `Job()` to create a job:

In [16]:
job = mdb.Job(name='Job-1', model='Model-1')

Then we can write the model to an input file (.inp):

In [17]:
job.writeInput()

Then we can submit the job:

In [18]:
job.submit()
job.waitForCompletion()

## Save the Abaqus model to a .cae file

We can use `saveAs()` to save the Abaqus model to a .cae file:

In [19]:
mdb.saveAs('compression.cae')

It should be noted that we have to use this function to save the model when we use abqpy to build an Abaqus model. It is because that when we execute all the above codes, the Python script has not been sent to Abaqus kernel. All the functions mentioned above are included in abqpy, however, nothing has been done inside these functions, they are just provided for type hints. Therefore, if we want to send the Python script to the Abaqus kernel, we have to use the Abaqus command like this:

```shell
abaqus cae -noGUI script.py
```